# Day 11

Santa's previous password expired, and he needs help choosing a new one.

To help him remember his new password after the old one expires, Santa has devised a method of coming up with a password based on the previous one. Corporate policy dictates that passwords must be exactly eight lowercase letters (for security reasons), so he finds his new password by incrementing his old password string repeatedly until it is valid.

Incrementing is just like counting with numbers: xx, xy, xz, ya, yb, and so on. Increase the rightmost letter one step; if it was z, it wraps around to a, and repeat with the next letter to the left until one doesn't wrap around.

Unfortunately for Santa, a new Security-Elf recently started, and he has imposed some additional password requirements:

    Passwords must include one increasing straight of at least three letters, like abc, bcd, cde, and so on, up to xyz. They cannot skip letters; abd doesn't count.
    Passwords may not contain the letters i, o, or l, as these letters can be mistaken for other characters and are therefore confusing.
    Passwords must contain at least two different, non-overlapping pairs of letters, like aa, bb, or zz.

For example:

    hijklmmn meets the first requirement (because it contains the straight hij) but fails the second requirement requirement (because it contains i and l).
    abbceffg meets the third requirement (because it repeats bb and ff) but fails the first requirement.
    abbcegjk fails the third requirement, because it only has one double letter (bb).
    The next password after abcdefgh is abcdffaa.
    The next password after ghijklmn is ghjaabcc, because you eventually skip all the passwords that start with ghi..., since i is not allowed.

Given Santa's current password (your puzzle input), what should his next password be?

## Puzzle 1

In [1]:
import string

import numpy as np

DOUBLES = [2 * _ for _ in string.ascii_lowercase]


def has_straight(pwd:str) -> bool:
    """Returns True if password has straight of 3 letters
    
    :param pwd: password string
    """
    diffs = np.diff([ord(_) for _ in pwd])
    for i in range(6):
        if diffs[i] == diffs[i+1] == 1:
            return True
    return False


def allowed_letters(pwd:str) -> bool:
    """Returns True if password only contains allowed letters"""
    for _ in "iol":
        if pwd.count(_):
            return False
    
    return True


def has_pairs(pwd:str) -> bool:
    """Returns True if password has >2 non-overlapping pairs
    
    :param pwd: password string
    """
    paircounts = {dbl: bool(pwd.count(dbl)) for dbl in DOUBLES}
    if sum(paircounts.values()) > 1:
        return True
    
    return False


def increment_letter(lttr:str) -> str:
    """Return next letter in sequence
    
    :param lttr: letter to increment
    """
    _ = ord(lttr)
    if _ in (104, 107, 110):
        return chr(_ + 2)
    elif _ == 122:
        return "a"
    else:
        return chr(_ + 1)


def increment_password(pwd:str) -> str:
    """Return next password candidate in sequence
    
    :param pwd: password to increment
    """
    rpwd = list(pwd[::-1])
    for idx, _ in enumerate(rpwd):
        next_lett = increment_letter(_)
        rpwd[idx] = next_lett
        if next_lett != "a":
            break
    return "".join(rpwd[::-1])
    

def skip_invalid_letters(pwd:str) -> str:
    """Move invalid password letters to next valid choice
    
    :param pwd: password to update
    """
    pwd = list(pwd)
    for idx, _ in enumerate(pwd):
        if _ in "iol":
            pwd[idx] = chr(ord(_) + 1)
            pwd[idx + 1:] = "a" * (len(pwd) - idx - 1)
            return "".join(pwd)
    return "".join(pwd)
    
    
def next_password(pwd:str) -> str:
    """Return next valid password in sequence
    
    :param pwd: password to increment
    """
    pwd = skip_invalid_letters(pwd)
    
    while True:
        next_pwd = increment_password(pwd)
        if has_straight(next_pwd) and has_pairs(next_pwd):
            return next_pwd
        pwd = next_pwd

In [2]:
pwds = ("hijklmmn",
        "abbceffg",
        "abbcegjk",
        "abcdefgh",
        "abcdffaa",
        "ghijklmn",
        "ghjaabcc")

for pwd in pwds:
    print(pwd, has_straight(pwd), allowed_letters(pwd), has_pairs(pwd), increment_password(pwd))
    
print(next_password("abcdefgh"))
print(next_password("ghijklmn"))

hijklmmn True False False hijklmmp
abbceffg False True True abbceffh
abbcegjk False True False abbcegjm
abcdefgh True True False abcdefgj
abcdffaa True True True abcdffab
ghijklmn True False False ghijklmp
ghjaabcc True True True ghjaabcd
abcdffaa
ghjaabcc


### Solution

In [3]:
with open("day11.txt", "r") as ifh:
    print(next_password(ifh.read().strip()))

cqjxxyzz


## Puzzle 2

Santa's password expired again. What's the next one?

In [4]:
next_password("cqjxxyzz")

'cqkaabcc'